In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkFiles


path = "../data/conditions_truncated.csv"

conf = SparkConf()
conf.getAll()

sc = SparkContext(appName="test")
    
#spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
spark = SparkSession(sc)
sc.setLogLevel("ERROR")

textfile = sc.textFile(path)

sc.addFile(path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 22:28:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [33]:
SUPPORT = 1000
K = 2

#def build_pairs(basket, filtered_diseases):
#
#    
#    basket_diseases = list(basket[1])
#
#    bigrams = list()
#
#    length = len(basket_diseases)
#
#    for k in range(length):
#        disease_1 = basket_diseases[k]
#        if disease_1 not in filtered_diseases:
#            continue
#        for i in range(k + 1, length):
#            disease_2 = basket_diseases[i]
#            if disease_2 not in filtered_diseases:
#                continue
#            bigrams.append((disease_1 + "," + disease_2, 1))
#    
#    return bigrams

def build_pairs(basket, filtered_diseases):
    basket_diseases = basket[1]
    bigrams = []

    for i, disease_1 in enumerate(basket_diseases):
        if disease_1 not in filtered_diseases:
            continue

        for disease_2 in basket_diseases[i+1:]:
            if disease_2 not in filtered_diseases:
                continue

            bigram = f"{disease_1},{disease_2}"
            bigrams.append((bigram, 1))
    
    return bigrams




def main():

    baskets = textfile.map(lambda line: line.split(",")).map(lambda pair: (pair[2],[pair[4]])).reduceByKey(lambda a,b: a + b)

    printable = baskets.collect()

    print(sc.parallelize(printable).count())

    first_support = baskets.flatMap(lambda basket: [(disease, 1) for disease in basket[1]]).reduceByKey(lambda a,b: a + b).filter(lambda line: line[1] > SUPPORT)

    printable2 = first_support.collect()

    print(sc.parallelize(printable2).count())

    filtered_diseases = first_support.map(lambda line: line[0]).collect()

    bigrams_support = baskets.flatMap(lambda basket: build_pairs(basket, filtered_diseases)).reduceByKey(lambda a,b: a + b).filter(lambda line: line[1] > SUPPORT)

    printable3 = bigrams_support.sortBy(lambda line: line[1], False).collect()

    for row in printable3:
        print(row)

    print(len(printable3))
    

if __name__ == "__main__":
    main()

21574
32
('444814009,444814009', 14159)
('72892002,72892002', 12460)
('444814009,195662009', 7360)
('195662009,444814009', 7162)
('162864005,444814009', 6800)
('15777000,444814009', 6527)
('271737000,444814009', 6138)
('444814009,10509002', 5794)
('10509002,444814009', 5719)
('59621000,444814009', 5654)
('72892002,444814009', 5535)
('444814009,72892002', 4930)
('15777000,271737000', 4929)
('40055000,444814009', 4510)
('195662009,195662009', 4211)
('19169002,444814009', 3850)
('162864005,195662009', 3830)
('15777000,195662009', 3690)
('271737000,195662009', 3422)
('195662009,10509002', 3336)
('10509002,195662009', 3237)
('59621000,195662009', 3190)
('72892002,195662009', 3170)
('162864005,10509002', 2894)
('195662009,72892002', 2839)
('15777000,10509002', 2803)
('59621000,72892002', 2784)
('271737000,10509002', 2650)
('40055000,195662009', 2601)
('10509002,10509002', 2491)
('59621000,10509002', 2485)
('72892002,10509002', 2459)
('10509002,72892002', 2320)
('19169002,72892002', 2276)
('1

In [ ]:
sc.stop()